In [1]:
import sys
sys.path.append('./model_construction/')

In [2]:

import logging
import time
from argparse import ArgumentParser

import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

import datasets
import utils
import model
from nni.nas.pytorch.utils import AverageMeter
from nni.retiarii import fixed_arch
import random



/usr/local/lib/python3.8/dist-packages/pandas/compat/_optional.py:149: UserWarning: Pandas requires version '1.3.1' or newer of 'bottleneck' (version '1.2.1' currently installed).
  warnings.warn(msg, UserWarning)
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
device = 'cuda'

In [4]:
def validate(mdl):
    top1 = AverageMeter("top1")
    top5 = AverageMeter("top5")
    losses = AverageMeter("losses")

    mdl.eval()

    with torch.no_grad():
        for step, (X, y) in enumerate(valid_loader):
            X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)
            bs = X.size(0)
            logits = mdl(X)
            accuracy = utils.accuracy(logits, y, topk=(1,))
            top1.update(accuracy["acc1"], bs)

           

    
    return top1.avg


In [5]:
_, dataset_valid = datasets.get_dataset("fmnist3", cutout_length=0)
valid_loader = torch.utils.data.DataLoader(dataset_valid,
                                               batch_size=64,
                                               shuffle=False,
                                               num_workers=4,
                                               pin_memory=True)


    
    

In [33]:
for x,y in valid_loader:
    break

In [30]:
with open('model_construction/darts_fmnist2_fmnist3/checkpoint.json') as inp:
        base = json.loads(inp.read())

In [31]:
with fixed_arch(base):
        cnn_model = model.CNN(28, 1, 16, 8 , 3, auxiliary=False, node_cls = model.OnesNode)

[2022-07-07 20:34:22] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switc

/usr/local/lib/python3.8/dist-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [45]:
import json
import tqdm
def prune(path_to_json, path_to_pt, percentage, out_json):
    with open(path_to_json) as inp:
        base = json.loads(inp.read())
    with fixed_arch(base):
        cnn_model = model.CNN(28, 1, 16, 8 , 3, auxiliary=False, node_cls = model.OnesNode)
        cnn_model.load_state_dict(torch.load(path_to_pt), strict=False)
        cnn_model = cnn_model.to(device)  
    gamma_values = {}
    crit = torch.nn.CrossEntropyLoss()
    for x,y in tqdm.tqdm_notebook(valid_loader):
        cnn_model.zero_grad()
        x = x.to(device)
        y = y.to(device)
        out = cnn_model(x)
        loss = crit(out, y)
        loss.backward()       
        for cell_id, cell in enumerate(cnn_model.cells):
            for op_id, op in enumerate(cell.mutable_ops):
                for gamma_id,grad_val in enumerate(op.gammas.grad.detach()):                    
                    if (cell_id, op_id, gamma_id) not in gamma_values:
                        gamma_values[(cell_id, op_id, gamma_id)] = abs(grad_val).item()
                    else:
                        gamma_values[(cell_id, op_id, gamma_id)] += abs(grad_val).item()
                        
    
    x = x.cpu()
        
    ordered_gammas = sorted(gamma_values.items(), key = lambda x:-x[1])
    ordered_gammas = [(g[1], g[0]) for g in ordered_gammas]    
    total_pruned = 0
    current_gamma_id = 0
    while total_pruned / len(ordered_gammas) < percentage/100:
        modified = base.copy()
        cell_id, op_id, gamma_id = ordered_gammas[current_gamma_id][1]
        cell_type_red = cnn_model.cells[cell_id].reduction
        if cell_type_red:
            name = f"reduce_n{op_id+2}_p{gamma_id}"
        else:
            name = f"normal_n{op_id+2}_p{gamma_id}"
        assert name in modified
        modified[name] = 'zero'
        with fixed_arch(modified):
            cnn_model = model.CNN(28, 1, 16, 8 , 3, auxiliary=False, node_cls = model.ZeroNode)
            cnn_model.load_state_dict(torch.load(path_to_pt), strict=False)
            cnn_model.eval()
            cnn_model = cnn_model
            out = nn.functional.softmax(cnn_model(x),dim=1)
            diff = abs(out[1][0] - out[0][0]).sum()
            if diff>1e-3:  
                print ('good', name, diff)
                
                base = modified
                total_pruned += 1
            else:
                print ('bad', name, diff)
                
            current_gamma_id += 1
    with open(out_json, 'w') as out:
        out.write(json.dumps(modified))

prune('./model_construction/darts_fmnist2_fmnist3/checkpoint.json',
     './model_construction/darts_fmnist2_fmnist3/optimized_darts.pt',
      25,
     './model_construction/molchanov/prune_25.json')

        

[2022-07-07 20:38:53] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switc

/tmp/ipykernel_74764/1808757506.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x,y in tqdm.tqdm_notebook(valid_loader):


  0%|          | 0/32 [00:00<?, ?it/s]

[2022-07-07 20:38:58] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1

good reduce_n5_p3 tensor(0.0669, grad_fn=<SumBackward0>)
[2022-07-07 20:39:00] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], '

In [47]:
prune('./model_construction/darts_fmnist2_fmnist3/checkpoint.json',
     './model_construction/darts_fmnist2_fmnist3/optimized_darts.pt',
      50,
     './model_construction/molchanov/prune_50.json')

prune('./model_construction/darts_fmnist2_fmnist3/checkpoint.json',
     './model_construction/darts_fmnist2_fmnist3/optimized_darts.pt',
      75,
     './model_construction/molchanov/prune_75.json')

                

[2022-07-07 20:39:59] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switc

/tmp/ipykernel_74764/2510379445.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x,y in tqdm.tqdm_notebook(valid_loader):


  0%|          | 0/32 [00:00<?, ?it/s]

[2022-07-07 20:40:03] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1

good reduce_n5_p3 tensor(0.0669, grad_fn=<SumBackward0>)
[2022-07-07 20:40:05] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], '

...

[2022-07-07 20:40:13] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1

good reduce_n5_p3 tensor(0.0669, grad_fn=<SumBackward0>)
[2022-07-07 20:40:14] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], '

bad normal_n2_p0 tensor(0., grad_fn=<SumBackward0>)
[2022-07-07 20:40:16] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'zero', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 

good reduce_n5_p3 tensor(0.0059, grad_fn=<SumBackward0>)
[2022-07-07 20:40:18] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'zero', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'zero', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_s

good reduce_n4_p0 tensor(0.0059, grad_fn=<SumBackward0>)
[2022-07-07 20:40:19] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'zero', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'zero', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch': [3, 2], 'redu

[2022-07-07 19:44:25] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switc

/tmp/ipykernel_74764/3512786028.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x,y in tqdm.tqdm_notebook(valid_loader):


  0%|          | 0/32 [00:00<?, ?it/s]

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

[2022-07-07 14:45:07] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switc

good reduce_n3_p2 tensor(0.0820, grad_fn=<SumBackward0>)
[2022-07-07 14:45:10] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], '

good normal_n2_p1 tensor(0.0441, grad_fn=<SumBackward0>)
[2022-07-07 14:45:13] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch':

good reduce_n3_p0 tensor(0.0441, grad_fn=<SumBackward0>)
[2022-07-07 14:45:16] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_

good reduce_n4_p1 tensor(0.0441, grad_fn=<SumBackward0>)
[2022-07-07 14:45:19] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch': [3, 2], 'redu

[2022-07-07 16:17:38] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switc

good reduce_n2_p0 tensor(0.0052, grad_fn=<SumBackward0>)
[2022-07-07 16:17:41] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': 

good reduce_n3_p2 tensor(0.6869, grad_fn=<SumBackward0>)
[2022-07-07 16:17:44] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_

good reduce_n4_p2 tensor(0.0135, grad_fn=<SumBackward0>)
[2022-07-07 16:17:47] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2

good normal_n5_p4 tensor(0.0459, grad_fn=<SumBackward0>)
[2022-07-07 16:17:50] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 

good reduce_n2_p0 tensor(0.4307, grad_fn=<SumBackward0>)
[2022-07-07 16:17:53] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_swi

good normal_n2_p0 tensor(0.0459, grad_fn=<SumBackward0>)
[2022-07-07 16:17:56] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switc

good reduce_n3_p0 tensor(0.0459, grad_fn=<SumBackward0>)
[2022-07-07 16:18:00] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'r

good reduce_n4_p1 tensor(0.0459, grad_fn=<SumBackward0>)
[2022-07-07 16:18:03] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch': [3, 

In [14]:
prune('./model_construction/darts_fmnist2/checkpoint.json',
     './model_construction/naive_dartslike_retrain_gs/retrained_gamma.pt',
      25,
     './model_construction/naive_dartslike_retrain_gs/prune_25.json')

prune('./model_construction/darts_fmnist2/checkpoint.json',
     './model_construction/naive_dartslike_retrain_gs/retrained_gamma.pt',
      50,
     './model_construction/naive_dartslike_retrain_gs/prune_50.json')

prune('./model_construction/darts_fmnist2/checkpoint.json',
     './model_construction/naive_dartslike_retrain_gs/retrained_gamma.pt',
      75,
     './model_construction/naive_dartslike_retrain_gs/prune_75.json')

[2022-07-07 17:54:13] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switc

good normal_n5_p0 tensor(0.3947, grad_fn=<SumBackward0>)
[2022-07-07 17:54:15] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], '

good reduce_n3_p2 tensor(0.3947, grad_fn=<SumBackward0>)
[2022-07-07 17:54:18] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'norm

good reduce_n4_p0 tensor(0.3947, grad_fn=<SumBackward0>)
[2022-07-07 17:54:21] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0]

good reduce_n5_p0 tensor(0.3947, grad_fn=<SumBackward0>)
[2022-07-07 17:54:24] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_sw

good normal_n4_p0 tensor(0.3947, grad_fn=<SumBackward0>)
[2022-07-07 17:54:26] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch

good reduce_n5_p0 tensor(0.3947, grad_fn=<SumBackward0>)
[2022-07-07 17:54:29] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce

good reduce_n5_p2 tensor(0.3947, grad_fn=<SumBackward0>)
[2022-07-07 17:54:32] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch':

good normal_n5_p3 tensor(0.6831, grad_fn=<SumBackward0>)
[2022-07-07 17:54:35] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch'

In [48]:
modified

{'normal_n2_p0': 'sepconv3x3',
 'normal_n2_p1': 'sepconv3x3',
 'normal_n3_p0': 'sepconv3x3',
 'normal_n3_p1': 'zero',
 'normal_n3_p2': 'sepconv5x5',
 'normal_n4_p0': 'sepconv5x5',
 'normal_n4_p1': 'sepconv3x3',
 'normal_n4_p2': 'maxpool',
 'normal_n4_p3': 'sepconv5x5',
 'normal_n5_p0': 'dilconv3x3',
 'normal_n5_p1': 'dilconv5x5',
 'normal_n5_p2': 'sepconv3x3',
 'normal_n5_p3': 'zero',
 'normal_n5_p4': 'dilconv5x5',
 'reduce_n2_p0': 'zero',
 'reduce_n2_p1': 'zero',
 'reduce_n3_p0': 'dilconv3x3',
 'reduce_n3_p1': 'zero',
 'reduce_n3_p2': 'zero',
 'reduce_n4_p0': 'sepconv5x5',
 'reduce_n4_p1': 'skipconnect',
 'reduce_n4_p2': 'zero',
 'reduce_n4_p3': 'zero',
 'reduce_n5_p0': 'maxpool',
 'reduce_n5_p1': 'dilconv5x5',
 'reduce_n5_p2': 'dilconv5x5',
 'reduce_n5_p3': 'zero',
 'reduce_n5_p4': 'zero',
 'normal_n2_switch': [1, 0],
 'normal_n3_switch': [2, 1],
 'normal_n4_switch': [3, 2],
 'normal_n5_switch': [2, 3],
 'reduce_n2_switch': [1, 0],
 'reduce_n3_switch': [2, 1],
 'reduce_n4_switch': [3

In [16]:
# sigmoid retrainю Note that sigmoids are non-binary!
model_eval('./model_construction/darts_fmnist2/checkpoint.json', './model_construction/out.pt')

[2022-07-07 12:59:36] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switc

0.9415

In [15]:
# fmnist3
model_eval('./model_construction/darts_fmnst3/checkpoint.json', './model_construction/darts_fmnst3/optimized_darts.pt')

[2022-07-02 23:28:31] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'sepconv3x3', 'normal_n3_p0': 'dilconv3x3', 'normal_n3_p1': 'sepconv3x3', 'normal_n3_p2': 'sepconv5x5', 'normal_n4_p0': 'sepconv3x3', 'normal_n4_p1': 'sepconv5x5', 'normal_n4_p2': 'dilconv5x5', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'sepconv3x3', 'normal_n5_p1': 'sepconv3x3', 'normal_n5_p2': 'dilconv5x5', 'normal_n5_p3': 'sepconv5x5', 'normal_n5_p4': 'dilconv3x3', 'reduce_n2_p0': 'sepconv3x3', 'reduce_n2_p1': 'dilconv5x5', 'reduce_n3_p0': 'sepconv5x5', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv3x3', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'sepconv5x5', 'reduce_n4_p2': 'dilconv3x3', 'reduce_n4_p3': 'skipconnect', 'reduce_n5_p0': 'dilconv3x3', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'skipconnect', 'reduce_n5_p3': 'skipconnect', 'reduce_n5_p4': 'skipconnect', 'normal_n2_switch': [0, 1], 'normal_n3_switch': [2, 0], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [3, 0], '

0.9055

In [6]:
for run in range(3):
    for perc in [25, 50, 75]:
        print (run, perc, model_eval(f'./model_construction/random_pruned/{run}_{perc}.json', f'./model_construction/random_pruned/{run}_{perc}.pt'))

[2022-07-04 22:00:36] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'zero', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 

2 50 0.919
[2022-07-04 22:00:51] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'zero', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'zero', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch': [3, 2], 'reduce_n5_switch': [4, 3]}
2 75 0.881
